In [1]:
import datetime
import itertools
import pandas as pd

from san.extras.strategy.strategy import Strategy

/home/alex/santiment/sanpy/san/__init__.py:31: UserWarning: The package sanpy is out of date. Your version is 0.8.5, the latest is 0.8.11.
  warn(warning)


# What is strategy

Strategy is a tool that allows you to create and backtest trading strategies based on signals. That means that any change in the portfolio is triggired by a signal.

# Key Components

## Portfolio

Portfolio is represented by assets and assets' shares in the portfolio on a particular date.\
Example:\
    - 2021-01-01: ethereum (80%), dai (20%)\
    - 2021-01-02: ethereum (50%), dai (50%)

Given approach makes it possible not to consider amount of money you allocate in the portfolio.

## Authorized Assets

Authorized assets are assets that could be included in the portfolio on a given date. If asset is authorized on 2021-01-01 that does not mean that the asset must be included in the portfolio on 2021-01-01 i.e. it's share in the portfolio could be 0.

There are 2 types of assets:
* Reserve assets
* Common assets

...
    

## Signals

Signal is an event that happens in a certain point in time. In general case the signal could be represented by a set of datetimes. A signal could correspond to a particular asset or to a few assets.

Signals could be interpreted in a different ways. We interpret signals in a 3 different ways:

1. Buy-signals \
    Buy-signal an instruction to buy a particluar asset. E.g. include a particular asset in the portfolio. In general case, increase asset's share in the portfolio.

2. Sell-signals \
    Sell-signal an instruction to sell a particluar asset. E.g. exclude a particular asset from the portfolio. In general case, decrease asset's share in the portfolio.


3. Rebalance-signals \
    Rebalance-signals leads to some changes in the portfolio structure. These changes may or may not lead to including an asset  or complete asset excluding from the portfolio.

## Prices

Prices are used to recompute assets' shares changes conditioned by price changes.

# Usage

## Init Strategy

In [2]:
index = Strategy(start_dt='2021-01-01', init_asset='dai')

## Adding/removing assets

In [3]:
# add reserve asset
index.assets.add(assets={'dai': ['2021-01-01', '2021-01-04']}, assets_type='r')
index.assets.reserve_assets

,asset
index,
2021-01-01,dai
2021-01-02,dai
2021-01-03,dai
2021-01-04,dai


In [4]:
# add non-reserve assets
index.assets.add({
    'ethereum': ['2021-01-01', '2021-01-02', '2021-01-04', '2021-01-04'],
    'uniswap': ['2021-01-01', '2021-01-04']
})
index.assets.assets

,asset
index,
2021-01-01,ethereum
2021-01-01,uniswap
2021-01-02,ethereum
2021-01-02,uniswap
2021-01-03,uniswap
2021-01-04,ethereum
2021-01-04,uniswap


In [5]:
# Remove uniswap for 2021-01-03 - 2021-01-05
index.assets.remove({'uniswap': ['2021-01-03', '2021-01-04']})
index.assets.assets

,asset
index,
2021-01-01,ethereum
2021-01-01,uniswap
2021-01-02,ethereum
2021-01-02,uniswap
2021-01-04,ethereum


## Adding/removing signals

In [6]:
buy_signals = pd.DataFrame({
    'dt': ['2021-01-05', '2021-01-05', '2021-02-10', '2021-03-10'],
    'asset': ['ethereum', 'uniswap', 'ethereum', 'uniswap']
})

buy_signals['trade_percantage'] = buy_signals.apply(lambda x: 0.5 if x['asset'] == 'ethereum' else 0.8, axis=1)
buy_signals['decision_delay'] = datetime.timedelta(days=2)

sell_signals_1 = pd.DataFrame({
    'dt': ['2021-02-05', '2021-03-15'],
    'asset': ['ethereum', 'uniswap']
})

sell_signals_2 = pd.DataFrame({
    'dt': ['2021-03-01'],
    'asset': ['uniswap']
})

In [7]:
# Add buy signals to the index
index.signals.add('b', buy_signals, signal_name='buy')
index.signals.buy_signals

,dt,signal,asset,trade_percantage,decision_delay
2021-01-07,2021-01-05,buy,ethereum,0.5,2 days
2021-01-07,2021-01-05,buy,uniswap,0.8,2 days
2021-02-12,2021-02-10,buy,ethereum,0.5,2 days
2021-03-12,2021-03-10,buy,uniswap,0.8,2 days


In [8]:
# Add sell signals to the index
index.signals.add('s', sell_signals_1, signal_name='sell_1')
index.signals.add('s', sell_signals_2, signal_name='sell_2')
index.signals.sell_signals

,dt,signal,asset,trade_percantage,decision_delay
2021-02-05,2021-02-05,sell_1,ethereum,1,0 days
2021-03-15,2021-03-15,sell_1,uniswap,1,0 days
2021-03-01,2021-03-01,sell_2,uniswap,1,0 days


In [9]:
# Remove part of the sell signals
index.signals.remove('s', signal_name='sell_2')
index.signals.sell_signals

,dt,signal,asset,trade_percantage,decision_delay
2021-02-05,2021-02-05,sell_1,ethereum,1,0 days
2021-03-15,2021-03-15,sell_1,uniswap,1,0 days


In [10]:
# Define rebalance signals df
rebalance_signals = pd.DataFrame({
    'dt': ['2021-02-01', '2021-03-01']
})

In [11]:
# Add rebalance signals
index.signals.add('r', rebalance_signals, signal_name='rebalance')
index.signals.rebalance_signals

,dt,signal,asset,trade_percantage,decision_delay
2021-02-01,2021-02-01,rebalance,NaN,1,0 days
2021-03-01,2021-03-01,rebalance,NaN,1,0 days


## Setting prices

In [12]:
prices = pd.DataFrame(
    list(itertools.product(['2021-01-01', '2021-01-02', '2021-01-03'], ['ethereum', 'dai', 'uniswap'])),
    columns=['dt', 'asset']
).set_index('dt')
prices['price'] = [4000, 1, 100, 4100, 1, 90, 4200, 1, 110]

In [13]:
index.prices.set(prices)

In [14]:
index.prices.prices.loc[['2021-01-01']]

,asset,price,price_change
dt,,,
2021-01-01,dai,1,1.0
2021-01-01,ethereum,4000,1.0
2021-01-01,uniswap,100,1.0


## Adding periodic rebalance

In [15]:
# add default monthly rebalancing
index.add_periodic_rebalance('0 0 1 * *', skip_rebalance_on_init=False)

## Building portfolio

In [16]:
index.build_portfolio('2021-01-01', '2021-01-03', rebuild=True)

In [17]:
index.portfolio

,asset,share
dt,,
2021-01-01,dai,1.0
2021-01-02,dai,1.0
2021-01-03,dai,1.0
